In [ ]:
# winequality-red.csv , winequality-white.csv파일의 데이터를 읽어서 테이블에 저장

# 3. 데이터베이스는 _2.ipynb에서 만든 data_repo 사용
# 4. 테이블 만들기 (red_wine -> wine)
# 5. 데이터 읽어서 red_wine 테이블에 저장
# 6. 7등급 이상의 데이터를 조회해서 파일로 저장 (high-quality-wine.csv)

In [ ]:
# 데이터베이스 연동 패키지(모듈) 준비
import pymysql
# 데이터 분석 패키지 준비
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# 1. 각 파일의 데이터를 읽어서 type 컬럼을 만들고 'red', 'white' 저장
df_red = pd.read_csv('data-files/winequality-red.csv', sep=";")
df_white = pd.read_csv('data-files/winequality-white.csv', sep=";")

print(df_red.columns, df_white.columns) # 컬럼 동일성 확인

df_red['type'] = 'red'
df_white['type'] = 'white'

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object') Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red


In [6]:
# 2. 두 파일의 데이터를 병합
df_combined = pd.concat([df_red, df_white])
print(df_red.shape, df_white.shape, df_combined.shape)

(1599, 13) (4898, 13) (6497, 13)


In [ ]:
try:
    conn = pymysql.connect(
        host= "127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )
    
    cursor= conn.cursor()

    sql = "drop table if exists wine"
    cursor.execute(sql)
    sql = """ create table if not exists wine
         ( 
              idx int primary key auto_increment,
              fixed_acidity float not null,
              volatile_acidity float not null,
              citric_acid float not null,
              residual_sugar float not null,
              chlorides float not null,
              free_sulfur_dioxide float not null,
              total_sulfur_dioxide float not null,
              density float not null,
              pH float not null,
              sulphates float not null,
              alcohol float not null,
              quality int not null,
              type varchar(10) not null
         )"""
    cursor.execute(sql)

except Exception as e:
    print("오류발생 : ", e)
finally:

    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [10]:
try:
    conn = pymysql.connect(
        host= "127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )

    cursor = conn.cursor()

    # 명령 실행
    sql = "delete from wine"
    cursor.execute(sql)

    sql = """insert into wine 
             (fixed_acidity, volatile_acidity, citric_acid, residual_sugar, chlorides, free_sulfur_dioxide, total_sulfur_dioxide, density, pH, sulphates, alcohol, quality, type)
             values 
             (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

    data = df_combined.values.tolist()
    cursor.executemany(sql, data)
    conn.commit()

except Exception as e:
    print("오류발생 : ", e)
    if conn:
        conn.rollback()

finally:
    if cursor:
        cursor.close()
    if conn:    
        conn.close()

In [ ]:
# 6. 7등급 이상의 데이터를 조회해서 파일로 저장 (high-quality-wine.csv)
try:
    conn = pymysql.connect(
        host= "127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )
    cursor = conn.cursor()

    sql = """select * from wine where quality >= %s"""
    cursor.execute(sql, [7])
    wine_data = cursor.fetchall()

    df_wine = pd.DataFrame(wine_data,  columns=['idx'] + list(df_combined.columns))
    df_wine.drop(['idx'], axis=1, inplace=True)
    df_wine.to_csv('data-files/high-quality-wine.csv', index=False)

except Exception as e:
    print("오류발생 : ", e)
finally:
    if cursor:
        cursor.close()
    if conn:    
        conn.close()

In [19]:
print(['idx'] + df_combined.columns)        # df_combined.columns 각 요소에 idx를 결합, 원하는게 아님
print(['idx'] + list(df_combined.columns))  # 두 리스트 병합

Index(['idxfixed acidity', 'idxvolatile acidity', 'idxcitric acid',
       'idxresidual sugar', 'idxchlorides', 'idxfree sulfur dioxide',
       'idxtotal sulfur dioxide', 'idxdensity', 'idxpH', 'idxsulphates',
       'idxalcohol', 'idxquality', 'idxtype'],
      dtype='object')
['idx', 'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality', 'type']
